# step_temperature with Two-Phase Detection

This notebook demonstrates how `step_temperature` automatically detects miscibility gaps (two-phase regions) in the Ag-Cu FCC system.

**Key concept:** At compositions inside the miscibility gap, `n_phases = 2`. Above the critical temperature, `n_phases = 1`.

In [ ]:
using OpenCALPHAD
using Plots

## 1. Load Database

In [ ]:
tdb_path = joinpath(@__DIR__, "..", "reftest", "tdb", "agcu.TDB")
db = read_tdb(tdb_path)
fcc = get_phase(db, "FCC_A1")

println("Phase: $(fcc.name)")
println("Constituents: $(fcc.constituents)")

## 2. Run step_temperature

At `x(Ag) = 0.5`, we are inside the miscibility gap at low temperatures.

In [ ]:
# Composition in two-phase region
x_Ag = 0.5

# Temperature range (crossing critical temperature ~1350K)
T_start = 1000.0
T_stop = 1500.0
T_step = 25.0

println("Composition: x(Ag) = $(x_Ag)")
println("Temperature range: $(T_start)K to $(T_stop)K")

In [ ]:
# Run step_temperature
result = step_temperature(fcc, db, x_Ag, T_start, T_stop, T_step)

println("Number of points: $(length(result.points))")

## 3. View Results

In [ ]:
# Display results as table
println("  T [K]  |  G [kJ/mol]  |  n_phases  |  Phase region")
println("  -------|--------------|------------|---------------")

for p in result.points
    region = p.n_phases == 1 ? "Single phase" : "Two-phase (miscibility gap)"
    println("  $(Int(p.axis_value))    |  $(round(p.gibbs_energy/1000, digits=2))       |     $(p.n_phases)      |  $(region)")
end

In [ ]:
# Find critical temperature (transition from 2-phase to 1-phase)
idx_critical = findfirst(p -> p.n_phases == 1, result.points)
if !isnothing(idx_critical) && idx_critical > 1
    T_critical = result.points[idx_critical].axis_value
    println("Critical temperature (approximate): $(T_critical)K")
else
    println("No transition found in this temperature range")
end

## 4. Plot Results

In [ ]:
# Extract data
T_values = [p.axis_value for p in result.points]
G_values = [p.gibbs_energy for p in result.points]
n_phases = [p.n_phases for p in result.points]

# Plot G vs T with phase regions
p1 = plot(T_values, G_values ./ 1000,
    xlabel="Temperature [K]",
    ylabel="Gibbs Energy [kJ/mol]",
    title="Ag-Cu FCC at x(Ag)=$(x_Ag)",
    linewidth=2, color=:blue,
    marker=:circle, markersize=4,
    legend=:topright, label="G(T)",
    size=(700, 500))

# Highlight two-phase region
two_phase_T = T_values[n_phases .== 2]
two_phase_G = G_values[n_phases .== 2]
if !isempty(two_phase_T)
    scatter!(p1, two_phase_T, two_phase_G ./ 1000,
        color=:red, markersize=6, label="Two-phase")
end

p1

In [ ]:
# Plot n_phases vs T
p2 = plot(T_values, n_phases,
    xlabel="Temperature [K]",
    ylabel="Number of phases",
    title="Phase count vs Temperature",
    linewidth=2, color=:green,
    marker=:square, markersize=4,
    ylims=(0.5, 2.5), yticks=[1, 2],
    legend=false, size=(700, 300))

hline!(p2, [1.5], linestyle=:dash, color=:gray)
annotate!(p2, T_start + 50, 2.2, text("Two-phase", 10, :left))
annotate!(p2, T_stop - 50, 1.2, text("Single phase", 10, :right))

p2

In [ ]:
# Combined plot
plot(p1, p2, layout=@layout([a; b{0.3h}]), size=(700, 700))

## 5. Interactive: Try Different Compositions

Compositions outside the miscibility gap will always show `n_phases = 1`.

In [ ]:
function analyze_composition(x_Ag_new)
    result_new = step_temperature(fcc, db, x_Ag_new, 1000.0, 1500.0, 50.0)
    
    T_vals = [p.axis_value for p in result_new.points]
    n_ph = [p.n_phases for p in result_new.points]
    
    n_twophase = count(==(2), n_ph)
    
    p = plot(T_vals, n_ph,
        xlabel="Temperature [K]",
        ylabel="n_phases",
        title="x(Ag) = $(x_Ag_new) ($(n_twophase) two-phase points)",
        linewidth=2, marker=:circle, markersize=4,
        ylims=(0.5, 2.5), yticks=[1, 2],
        legend=false, size=(500, 300))
    
    return p
end

# Compare different compositions
x_values = [0.1, 0.3, 0.5, 0.9]
plots = [analyze_composition(x) for x in x_values]
plot(plots..., layout=(2, 2), size=(900, 600))

## 6. Verify with Miscibility Gap Boundaries

Check that the phase boundaries from `find_miscibility_gap` match.

In [ ]:
println("Miscibility gap boundaries at various temperatures:")
println("  T [K]  |  x1 (Cu-rich)  |  x2 (Ag-rich)  |  x=0.5 inside?")
println("  -------|----------------|----------------|----------------")

for T in [1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0]
    gap = find_miscibility_gap(fcc, T, db)
    if isnothing(gap)
        println("  $(Int(T))    |  No gap        |  No gap        |  N/A")
    else
        inside = gap.x1 < 0.5 < gap.x2 ? "Yes" : "No"
        println("  $(Int(T))    |  $(round(gap.x1, digits=3))          |  $(round(gap.x2, digits=3))          |  $(inside)")
    end
end

## Summary

`step_temperature` automatically detects two-phase regions:

- **`n_phases = 2`**: Inside miscibility gap (common tangent construction)
- **`n_phases = 1`**: Single-phase solid solution

The Gibbs energy reported for two-phase regions is the equilibrium value from the common tangent, not the metastable single-phase value.